In [ ]:
import import_ipynb
import torch
import gpytorch 
import math
def Inference(Xtrain,Ytrain,likelihoods,models, training_iter=None,learning_rate= None,ifPrint = True,numPrint = 20):
    N = len(likelihoods) # number of dimension
    W = torch.zeros(N,N) # create the weighted adjacency matrix W
    
    if training_iter is None:
        training_iter = 200 # defauld training iter
    if learning_rate is None:
        learning_rate = 0.005 # default learning rate

    
    for n in range(N):
        print("processing for nodes" + str(n+1))
        
        # target variable for each node
        ytrain = Ytrain[:,n]
        
        # model for each nodes
        llh = likelihoods[n]
        model = models[n]  
        
        # in training mode
        model.train()
        llh.train()

        # optimizer
        optimizer = torch.optim.LBFGS(model.parameters(),lr = learning_rate)
        
        #marginal likelihood
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(llh,model)
    
        for i in range(training_iter):
            def closure():
                optimizer.zero_grad()
                output = model(Xtrain)
                loss = -mll(output, ytrain)
                loss.backward()
                return loss
            optimizer.step(closure)
            loss = optimizer.step(closure)
            if ifPrint is True:
                if (i+1) % numPrint == 0:
                    log_message = 'Iter %d/%d loss: %.3f' % (i + 1, training_iter, loss.item())
                    lengthscale_values = model.covar_module.base_kernel.lengthscale[0]   
                    for idx, lengthscale in enumerate(lengthscale_values):
                        log_message += ' W%d: %.8f' % (idx + 1, math.log(1 / lengthscale.item()))
                    noise_value = model.likelihood.noise.item()
                    log_message += ' noise: %.8f' % noise_value
                    print(log_message)
        W[n,:] = torch.log(1/model.covar_module.base_kernel.lengthscale[0])
    return W